# 1. Import needed libraries

In [3]:
import os
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
#---------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
#---------------------------------------
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#---------------------------------------
import warnings
warnings.filterwarnings("ignore")

# 2. Preprocessing

## 2.1 Load data

In [2]:
'''class_dict = tr_gen.class_indices
classes = list(class_dict.keys())
images, labels = next(ts_gen)

plt.figure(figsize=(20, 20))

for i, (image, label) in enumerate(zip(images, labels)):
    plt.subplot(4,4, i + 1)
    plt.imshow(image)
    class_name = classes[np.argmax(label)]
    plt.title(class_name, color='k', fontsize=15)

plt.show()'''

"class_dict = tr_gen.class_indices\nclasses = list(class_dict.keys())\nimages, labels = next(ts_gen)\n\nplt.figure(figsize=(20, 20))\n\nfor i, (image, label) in enumerate(zip(images, labels)):\n    plt.subplot(4,4, i + 1)\n    plt.imshow(image)\n    class_name = classes[np.argmax(label)]\n    plt.title(class_name, color='k', fontsize=15)\n\nplt.show()"

# 3. Building Deep Learning Model

In [4]:
img_rows, img_cols = 128, 128
input_shape = (img_rows, img_cols, 3)

#n_classes = df['category'].nunique()
n_classes = 4
print('Total number of unique categories:', n_classes)

from os import listdir, makedirs
from os.path import isfile, join, basename, splitext, isfile, exists

import numpy as np
import pandas as pd

from tqdm import tqdm_notebook

import tensorflow as tf
import keras.backend as K

import keras
from keras.models import Sequential, Model
from keras.layers import Dropout, Dense, Flatten, BatchNormalization
from keras.layers import DepthwiseConv2D, SeparableConvolution2D, Convolution2D, Conv2D,GRU, LSTM, AlphaDropout, Embedding, ZeroPadding2D,AveragePooling2D, MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D, Dropout
from keras.layers import Concatenate, Average, Maximum, Bidirectional, TimeDistributed
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
#from keras.engine.input_layer import Input
from keras.models import load_model
#from keras.initializers import LecunNormal

import matplotlib.pyplot as plt
import seaborn as sns

#pd.set_option('precision', 30)
np.set_printoptions(precision = 30)


#tf.set_random_seed(1090)

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as img

import cv2
import itertools
import pathlib
import warnings
from PIL import Image
from random import randint
warnings.filterwarnings('ignore')

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef as MCC
from sklearn.metrics import balanced_accuracy_score as BAS
from sklearn.metrics import classification_report, confusion_matrix


from tensorflow import keras
from keras import layers
import tensorflow as tf
#import tensorflow_addons as tfa
from tensorflow.keras.preprocessing import image_dataset_from_directory
##from keras.utils.vis_utils import plot_model
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG
from tensorflow.keras.layers import SeparableConv2D, BatchNormalization, GlobalAveragePooling2D

from distutils.dir_util import copy_tree, remove_tree

import os
#print(os.listdir("../input/alzheimer-mri-dataset/Dataset"))
import tensorflow as tf
from keras.datasets import mnist
import cv2
import os
import pathlib
from keras.layers import Conv2D, Conv2DTranspose,Concatenate, Dropout, Dense, Reshape, LayerNormalization, LeakyReLU
from keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import f1_score, recall_score, precision_score
print("TensorFlow Version:", tf.__version__)

Total number of unique categories: 4
TensorFlow Version: 2.15.0


dataset 3: https://www.kaggle.com/datasets/plameneduardo/sarscov2-ctscan-dataset

In [5]:
class ReadDataset:
    def __init__(self, datasetpath, labels, image_shape):
        self.datasetpath = datasetpath
        self.labels = labels
        self.image_shape = image_shape
    def returListImages(self,):
        self.images = []
        for label in self.labels:
            self.images.append(list(pathlib.Path(os.path.join(self.datasetpath,
                                                              label)).glob('*.*')))
    def readImages(self,):
        self.returListImages()
        self.finalImages = []
        labels = []
        for label in range(len(self.labels)):
            for img in self.images[label]:
                img = cv2.imread(str(img))
                img = cv2.resize(img , self.image_shape)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img  = img/255
                self.finalImages.append(img)
                labels.append(label)
        images = np.array(self.finalImages)
        labels = np.array(labels)
        return images, labels
#/kaggle/input/alzheimer-mri-dataset/Dataset
readDatasetObject = ReadDataset('/kaggle/input/sarscov2-ctscan-dataset',
                               ['COVID', 'non-COVID'],
                               (128,128))
images_train_covid_ct, labels_train_covid_ct = readDatasetObject.readImages()
images_train_covid_ct.shape, labels_train_covid_ct.shape

((2481, 128, 128, 3), (2481,))

In [6]:
from sklearn.model_selection import train_test_split

X_train_covid_ct, X_test_covid_ct, y_train_covid_ct, y_test_covid_ct = train_test_split(
    images_train_covid_ct, labels_train_covid_ct, test_size=0.2, random_state=42)

In [7]:
X_train_covid_ct.shape, X_test_covid_ct.shape, y_train_covid_ct.shape, y_test_covid_ct.shape

((1984, 128, 128, 3), (497, 128, 128, 3), (1984,), (497,))

## Augmenting the data to match samples size with the dataset 1 and 2

In [8]:
import numpy as np
import cv2

def rotate_image(image, angle):
    """
    Rotate the image by the specified angle.
    """
    center = tuple(np.array(image.shape[1::-1]) / 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_image = cv2.warpAffine(image, rotation_matrix, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return rotated_image

def translate_image(image, tx, ty):
    """
    Translate the image by the specified translation parameters.
    """
    translation_matrix = np.float32([[1, 0, tx], [0, 1, ty]])
    translated_image = cv2.warpAffine(image, translation_matrix, image.shape[1::-1])
    return translated_image

# Example data
#X_train = np.random.rand(100, 28, 28)  # Assuming 100 images of size 28x28
#y_train = np.random.randint(0, 10, 100)  # Assuming 100 labels

# Augmentation parameters
rotation_angles = [20]
translations = [(5, 5)]

augmented_X_train = []
augmented_y_train = []

for image, label in zip(X_train_covid_ct, y_train_covid_ct):
    # Original image
    augmented_X_train.append(image)
    augmented_y_train.append(label)

    # Augment with rotations
    for angle in rotation_angles:
        rotated_image = rotate_image(image, angle)
        augmented_X_train.append(rotated_image)
        augmented_y_train.append(label)

    # Augment with translations
    for tx, ty in translations:
        translated_image = translate_image(image, tx, ty)
        augmented_X_train.append(translated_image)
        augmented_y_train.append(label)

# Convert lists to numpy arrays
augmented_X_train = np.array(augmented_X_train)
augmented_y_train = np.array(augmented_y_train)

# Shuffle the data
shuffle_indices = np.random.permutation(len(augmented_X_train))
augmented_X_train = augmented_X_train[shuffle_indices]
augmented_y_train = augmented_y_train[shuffle_indices]
augmented_X_train.shape, augmented_y_train.shape
# Now, augmented_X_train and augmented_y_train contain the augmented dataset.

((5952, 128, 128, 3), (5952,))

In [9]:

random_indices = np.random.choice(5952, 3728, replace=False)
augmented_X_train = augmented_X_train[random_indices]
augmented_y_train = augmented_y_train[random_indices]
augmented_X_train.shape, augmented_y_train.shape

((3728, 128, 128, 3), (3728,))

In [10]:
augmented_X_train.shape, augmented_y_train.shape

((3728, 128, 128, 3), (3728,))

In [11]:
labels_train_covid_ct = np.concatenate((y_train_covid_ct, augmented_y_train), axis=0)
images_train_covid_ct = np.concatenate((X_train_covid_ct, augmented_X_train), axis=0)
images_train_covid_ct.shape, labels_train_covid_ct.shape


((5712, 128, 128, 3), (5712,))

In [12]:
'''
X_val.shape, X_test.shape, y_val.shape, y_test.shape
'''

from sklearn.preprocessing import OneHotEncoder
labels_train_covid_ct = labels_train_covid_ct.reshape(-1, 1)
encoder = OneHotEncoder(sparse=False)

# Fit and transform the data
labels_train_covid_ct = encoder.fit_transform(labels_train_covid_ct)
labels_train_covid_ct.shape

(5712, 2)

In [13]:
y_test_covid_ct = np.concatenate((y_test_covid_ct, y_test_covid_ct, y_test_covid_ct), axis=0)
X_test_covid_ct = np.concatenate((X_test_covid_ct, X_test_covid_ct, X_test_covid_ct), axis=0)
X_test_covid_ct.shape, y_test_covid_ct.shape


((1491, 128, 128, 3), (1491,))

In [14]:
random_indices = np.random.choice(1491, 1311, replace=False)
X_test_covid_ct = X_test_covid_ct[random_indices]
y_test_covid_ct = y_test_covid_ct[random_indices]
X_test_covid_ct.shape, y_test_covid_ct.shape

((1311, 128, 128, 3), (1311,))

In [15]:
from sklearn.model_selection import train_test_split

X_val_covid_ct, X_test_covid_ct, y_val_covid_ct, y_test_covid_ct = train_test_split(
    X_test_covid_ct, y_test_covid_ct, test_size=0.5, random_state=42)

X_val_covid_ct.shape, X_test_covid_ct.shape, y_val_covid_ct.shape, y_test_covid_ct.shape


((655, 128, 128, 3), (656, 128, 128, 3), (655,), (656,))

In [16]:
'''
X_val.shape, X_test.shape, y_val.shape, y_test.shape
'''

from sklearn.preprocessing import OneHotEncoder
y_test_covid_ct = y_test_covid_ct.reshape(-1, 1)
encoder = OneHotEncoder(sparse=False)

# Fit and transform the data
y_test_covid_ct = encoder.fit_transform(y_test_covid_ct)
y_test_covid_ct.shape

(656, 2)

In [17]:
'''
X_val.shape, X_test.shape, y_val.shape, y_test.shape
'''

from sklearn.preprocessing import OneHotEncoder
y_val_covid_ct = y_val_covid_ct.reshape(-1, 1)
encoder = OneHotEncoder(sparse=False)

# Fit and transform the data
y_val_covid_ct = encoder.fit_transform(y_val_covid_ct)
y_val_covid_ct.shape

(655, 2)

## dataset 4: https://www.kaggle.com/datasets/sid321axn/covid-cxr-image-dataset-research

In [18]:
readDatasetObject = ReadDataset('/kaggle/input/covid-cxr-image-dataset-research/COVID_IEEE',
                               ['covid', 'normal', 'virus'],
                               (128,128))
images_train_covid_cxr, labels_train_covid_cxr = readDatasetObject.readImages()
images_train_covid_cxr.shape, labels_train_covid_cxr.shape

((1823, 128, 128, 3), (1823,))

In [19]:
from sklearn.model_selection import train_test_split

X_train_covid_cxr, X_test_covid_cxr, y_train_covid_cxr, y_test_covid_cxr = train_test_split(
    images_train_covid_cxr, labels_train_covid_cxr, test_size=0.2, random_state=1)

X_train_covid_cxr.shape, X_test_covid_cxr.shape, y_train_covid_cxr.shape, y_test_covid_cxr.shape

((1458, 128, 128, 3), (365, 128, 128, 3), (1458,), (365,))

## Augmenting the data to match samples size with the dataset 1 and 2

In [20]:
import numpy as np
import cv2

def rotate_image(image, angle):
    """
    Rotate the image by the specified angle.
    """
    center = tuple(np.array(image.shape[1::-1]) / 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_image = cv2.warpAffine(image, rotation_matrix, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return rotated_image

def translate_image(image, tx, ty):
    """
    Translate the image by the specified translation parameters.
    """
    translation_matrix = np.float32([[1, 0, tx], [0, 1, ty]])
    translated_image = cv2.warpAffine(image, translation_matrix, image.shape[1::-1])
    return translated_image

# Example data
#X_train = np.random.rand(100, 28, 28)  # Assuming 100 images of size 28x28
#y_train = np.random.randint(0, 10, 100)  # Assuming 100 labels

# Augmentation parameters
rotation_angles = [5]
translations = [(5, 5)]

augmented_X_train = []
augmented_y_train = []

for image, label in zip(X_train_covid_cxr, y_train_covid_cxr):
    # Original image
    augmented_X_train.append(image)
    augmented_y_train.append(label)

    # Augment with rotations
    for angle in rotation_angles:
        rotated_image = rotate_image(image, angle)
        augmented_X_train.append(rotated_image)
        augmented_y_train.append(label)

    # Augment with translations
    for tx, ty in translations:
        translated_image = translate_image(image, tx, ty)
        augmented_X_train.append(translated_image)
        augmented_y_train.append(label)

# Convert lists to numpy arrays
augmented_X_train = np.array(augmented_X_train)
augmented_y_train = np.array(augmented_y_train)

# Shuffle the data
shuffle_indices = np.random.permutation(len(augmented_X_train))
augmented_X_train = augmented_X_train[shuffle_indices]
augmented_y_train = augmented_y_train[shuffle_indices]
augmented_X_train.shape, augmented_y_train.shape
# Now, augmented_X_train and augmented_y_train contain the augmented dataset.

((4374, 128, 128, 3), (4374,))

In [21]:
random_indices = np.random.choice(4374, 4254, replace=False)
augmented_X_train = augmented_X_train[random_indices]
augmented_y_train = augmented_y_train[random_indices]
augmented_X_train.shape, augmented_y_train.shape

((4254, 128, 128, 3), (4254,))

In [22]:
#X_train_covid_cxr.shape, X_test_covid_cxr.shape, y_train_covid_cxr.shape, y_test_covid_cxr.shape

labels_train_covid_cxr = np.concatenate((y_train_covid_cxr, augmented_y_train), axis=0)
images_train_covid_cxr = np.concatenate((X_train_covid_cxr, augmented_X_train), axis=0)
images_train_covid_cxr.shape, labels_train_covid_cxr.shape


((5712, 128, 128, 3), (5712,))

In [23]:
'''
X_val.shape, X_test.shape, y_val.shape, y_test.shape
'''

from sklearn.preprocessing import OneHotEncoder
labels_train_covid_cxr = labels_train_covid_cxr.reshape(-1, 1)
encoder = OneHotEncoder(sparse=False)

# Fit and transform the data
labels_train_covid_cxr = encoder.fit_transform(labels_train_covid_cxr)
labels_train_covid_cxr.shape

(5712, 3)

In [24]:
#X_train_covid_cxr.shape, X_test_covid_cxr.shape, y_train_covid_cxr.shape, y_test_covid_cxr.shape
y_test_covid_cxr = np.concatenate((y_test_covid_cxr, y_test_covid_cxr, y_test_covid_cxr, y_test_covid_cxr), axis=0)
X_test_covid_cxr = np.concatenate((X_test_covid_cxr, X_test_covid_cxr, X_test_covid_cxr, X_test_covid_cxr), axis=0)
X_test_covid_cxr.shape, y_test_covid_cxr.shape


((1460, 128, 128, 3), (1460,))

In [25]:
random_indices = np.random.choice(1460, 1311, replace=False)
X_test_covid_cxr = X_test_covid_cxr[random_indices]
y_test_covid_cxr = y_test_covid_cxr[random_indices]
X_test_covid_cxr.shape, y_test_covid_cxr.shape


((1311, 128, 128, 3), (1311,))

In [26]:
from sklearn.model_selection import train_test_split
#
X_val_covid_cxr, X_test_covid_cxr, y_val_covid_cxr, y_test_covid_cxr = train_test_split(X_test_covid_cxr, y_test_covid_cxr, 
                                                test_size=0.5, random_state=42)
X_val_covid_cxr.shape, X_test_covid_cxr.shape, y_val_covid_cxr.shape, y_test_covid_cxr.shape


((655, 128, 128, 3), (656, 128, 128, 3), (655,), (656,))

In [27]:
'''
X_val.shape, X_test.shape, y_val.shape, y_test.shape
'''

from sklearn.preprocessing import OneHotEncoder
y_val_covid_cxr = y_val_covid_cxr.reshape(-1, 1)
encoder = OneHotEncoder(sparse=False)

# Fit and transform the data
y_val_covid_cxr = encoder.fit_transform(y_val_covid_cxr)
y_val_covid_cxr.shape

(655, 3)

In [28]:
y_val_covid_cxr.shape

(655, 3)

In [29]:
'''
X_val.shape, X_test.shape, y_val.shape, y_test.shape
'''

from sklearn.preprocessing import OneHotEncoder
y_test_covid_cxr = y_test_covid_cxr.reshape(-1, 1)
encoder = OneHotEncoder(sparse=False)

# Fit and transform the data
y_test_covid_cxr = encoder.fit_transform(y_test_covid_cxr)
y_test_covid_cxr.shape

(656, 3)

## Teacher 2 model of the Proposed MMF-MTL-TS approach

## GSCA layer

In [30]:
import tensorflow as tf
from tensorflow.keras import layers

class DeeperGlobalLocalAttentionLayer(layers.Layer):
    def __init__(self, units, activation='sigmoid', dropout_rate=0.2, use_scale=True, **kwargs):
        super(DeeperGlobalLocalAttentionLayer, self).__init__(**kwargs)
        self.units = units
        self.activation = activation
        self.dropout_rate = dropout_rate
        self.use_scale = use_scale

    def build(self, input_shape):
        _, _, _, channels = input_shape
        self.global_avg_pooling = layers.GlobalAveragePooling2D()
        self.global_attention = layers.Dense(units=self.units, activation=self.activation)
        
        self.local_conv1 = layers.Conv2D(filters=self.units, kernel_size=(1, 1), activation=self.activation)
        self.local_conv2 = layers.Conv2D(filters=self.units, kernel_size=(1, 1), activation=self.activation)
        
        if self.use_scale:
            self.global_scale = self.add_weight(shape=(1, 1, 1, 1), initializer='ones', trainable=True, name='global_scale')
            self.local_scale = self.add_weight(shape=(1, 1, 1, self.units), initializer='ones', trainable=True, name='local_scale')
        
        super(DeeperGlobalLocalAttentionLayer, self).build(input_shape)

    def call(self, inputs, training=None):
        # Deeper Global Attention
        global_avg = self.global_avg_pooling(inputs)
        global_attention = self.global_attention(global_avg)
        global_attention = tf.expand_dims(tf.expand_dims(global_attention, 1), 1)

        # Deeper Spatial Attention
        local_attention1 = self.local_conv1(inputs)
        local_attention1 = tf.reduce_mean(local_attention1, axis=[1, 2], keepdims=True)  # Reduce spatial dimensions
        local_attention2 = self.local_conv2(local_attention1)
        local_attention = tf.reduce_mean(local_attention2, axis=[1, 2], keepdims=True)  # Reduce spatial dimensions

        # Scale Global and Spatial Attention
        if self.use_scale:
            global_attention *= self.global_scale
            local_attention *= self.local_scale

        # Combine Global and Spatial Attention
        attention = tf.sigmoid(global_attention + local_attention)
        return attention

    def get_config(self):
        config = super(DeeperGlobalLocalAttentionLayer, self).get_config()
        config.update({'units': self.units, 'activation': self.activation, 'dropout_rate': self.dropout_rate,
                       'use_scale': self.use_scale})
        return config

class DeeperAttentionLayer(layers.Layer):
    def __init__(self, units=64, use_scale=True, **kwargs):
        super(DeeperAttentionLayer, self).__init__(**kwargs)
        self.units = units
        self.use_scale = use_scale

    def build(self, input_shape):
        _, H, W, C = input_shape
        ## add channel wise weights to performa as channel attention
        self.alpha = self.add_weight(shape=(1, 1, 1, C), initializer='ones', trainable=True, name='alpha')
        self.deeper_global_local_attention = DeeperGlobalLocalAttentionLayer(units=self.units, activation='sigmoid', 
                                                                              dropout_rate=0.2,  # You can adjust the dropout rate
                                                                              use_scale=self.use_scale)
        super(DeeperAttentionLayer, self).build(input_shape)

    def call(self, inputs, training=None):
        attention = self.deeper_global_local_attention(inputs, training=training)
        attention_feature = inputs * attention * self.alpha
        return attention_feature

    def get_config(self):
        config = super(DeeperAttentionLayer, self).get_config()
        config.update({'units': self.units, 'use_scale': self.use_scale})
        return config


In [31]:
'''from tensorflow.keras.models import load_model

# Define custom objects dictionary
custom_objects = {
    'DeeperAttentionLayer': DeeperAttentionLayer,
    ##'distillation_loss': distillation_loss
}
import shutil

# Source path of the model file (read-only directory)
source_path = '/kaggle/input/best-teacher-model-brain/best_teacher1_model_brain.keras'

# Destination path in the writable directory
destination_path = '/kaggle/working/best_teacher1_model_brain.keras'

# Copy the model file from the source path to the destination path
shutil.copyfile(source_path, destination_path)

# Load the model with custom objects
model1 = load_model('/kaggle/working/best_teacher1_model_brain.keras', 
                    custom_objects={'DeeperAttentionLayer':DeeperAttentionLayer})

# Now you can use the loaded model for evaluation or further training
model1.evaluate([images_test, images_test_ct], [labels_test_one_hot, labels_test_ct_one_hot])'''

"from tensorflow.keras.models import load_model\n\n# Define custom objects dictionary\ncustom_objects = {\n    'DeeperAttentionLayer': DeeperAttentionLayer,\n    ##'distillation_loss': distillation_loss\n}\nimport shutil\n\n# Source path of the model file (read-only directory)\nsource_path = '/kaggle/input/best-teacher-model-brain/best_teacher1_model_brain.keras'\n\n# Destination path in the writable directory\ndestination_path = '/kaggle/working/best_teacher1_model_brain.keras'\n\n# Copy the model file from the source path to the destination path\nshutil.copyfile(source_path, destination_path)\n\n# Load the model with custom objects\nmodel1 = load_model('/kaggle/working/best_teacher1_model_brain.keras', \n                    custom_objects={'DeeperAttentionLayer':DeeperAttentionLayer})\n\n# Now you can use the loaded model for evaluation or further training\nmodel1.evaluate([images_test, images_test_ct], [labels_test_one_hot, labels_test_ct_one_hot])"

In [32]:
'''from tensorflow.keras.models import load_model

# Define custom objects dictionary
custom_objects = {
    'DeeperAttentionLayer': DeeperAttentionLayer,
    ##'distillation_loss': distillation_loss
}
import shutil

# Source path of the model file (read-only directory)
source_path = '/kaggle/input/best-teacher-model-brain/best_teacher1_model_brain.keras'

# Destination path in the writable directory
destination_path = '/kaggle/working/best_teacher1_model_brain.keras'

# Copy the model file from the source path to the destination path
shutil.copyfile(source_path, destination_path)

# Load the model with custom objects
model1 = load_model('/kaggle/working/best_teacher1_model_brain.keras', 
                    custom_objects={'DeeperAttentionLayer':DeeperAttentionLayer})

# Now you can use the loaded model for evaluation or further training
model1.evaluate([images_test, images_test_ct], [labels_test_one_hot, labels_test_ct_one_hot])
'''

"from tensorflow.keras.models import load_model\n\n# Define custom objects dictionary\ncustom_objects = {\n    'DeeperAttentionLayer': DeeperAttentionLayer,\n    ##'distillation_loss': distillation_loss\n}\nimport shutil\n\n# Source path of the model file (read-only directory)\nsource_path = '/kaggle/input/best-teacher-model-brain/best_teacher1_model_brain.keras'\n\n# Destination path in the writable directory\ndestination_path = '/kaggle/working/best_teacher1_model_brain.keras'\n\n# Copy the model file from the source path to the destination path\nshutil.copyfile(source_path, destination_path)\n\n# Load the model with custom objects\nmodel1 = load_model('/kaggle/working/best_teacher1_model_brain.keras', \n                    custom_objects={'DeeperAttentionLayer':DeeperAttentionLayer})\n\n# Now you can use the loaded model for evaluation or further training\nmodel1.evaluate([images_test, images_test_ct], [labels_test_one_hot, labels_test_ct_one_hot])\n"

In [33]:
def RGSCA(x, filters, strides=(1, 1), use_projection=False):
    shortcut = x

    # Define the first convolutional layer of the block
    x = Conv2D(filters=filters, kernel_size=(3, 3), strides=strides, padding='same', 
               #activation = 'relu'
              )(x)
    x = DeeperAttentionLayer(units=filters, use_scale=True)(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    # Define the second convolutional layer of the block
    x = Conv2D(filters=filters, kernel_size=(3, 3), padding='same')(x)
    x = DeeperAttentionLayer(units=filters, use_scale=True)(x)
    
    x = BatchNormalization()(x)

    # If the stride is not (1, 1), the dimensions need to be adjusted
    if strides != (1, 1) or use_projection:
        
        shortcut = Conv2D(filters=filters, kernel_size=(1, 1), strides=strides, padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)

    # Add the shortcut (identity connection)
    
    x = tf.keras.layers.add([x, shortcut])
    
    x = tf.keras.layers.Activation('relu')(x)
    return x

#def build_resnet18(input_shape=(128, 128, 3), num_classes=2):
input_shape=(128, 128, 3)
inputs = Input(shape=input_shape)
inputs2 = Input(shape=input_shape)
#input_data = Input(shape=input_shape, name='input_data')
# Initial convolutional layer


x = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2), padding='same')(inputs)
x = DeeperAttentionLayer(units=64, use_scale=True)(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

x = RGSCA(x, filters=64)
x = DeeperAttentionLayer(units=64, use_scale=True)(x) ## GSCA layer

x = RGSCA(x, filters=64)
x = tf.keras.layers.Dropout(0.2)(x)

x = RGSCA(x, filters=64)
x = DeeperAttentionLayer(units=64, use_scale=True)(x) ## GSCA layer

x = RGSCA(x, filters=128, strides=(2, 2), use_projection=True)
x = DeeperAttentionLayer(units=128, use_scale=True)(x)

x = RGSCA(x, filters=128)
x = tf.keras.layers.Dropout(0.2)(x)

x = RGSCA(x, filters=128)
x = DeeperAttentionLayer(units=128, use_scale=True)(x) ## GSCA layer

x = RGSCA(x, filters=256, strides=(2, 2), use_projection=True)
x = DeeperAttentionLayer(units=256, use_scale=True)(x)

x = RGSCA(x, filters=256)
x = tf.keras.layers.Dropout(0.2)(x)

x = RGSCA(x, filters=256)
x = DeeperAttentionLayer(units=256, use_scale=True)(x) ## GSCA layer

x = RGSCA(x, filters=512, strides=(2, 2), use_projection=True)
x = DeeperAttentionLayer(units=512, use_scale=True)(x)

x = RGSCA(x, filters=512)
x = DeeperAttentionLayer(units=512, use_scale=True)(x)


x2 = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2), padding='same')(inputs2)
x2 = DeeperAttentionLayer(units=64, use_scale=True)(x2)
x2 = BatchNormalization()(x2)
x2 = tf.keras.layers.Activation('relu')(x2)
x2 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x2)

x2 = RGSCA(x2, filters=64)
x2 = DeeperAttentionLayer(units=64, use_scale=True)(x2) ## GSCA layer

x2 = RGSCA(x2, filters=64)
x2 = tf.keras.layers.Dropout(0.2)(x2)

x2 = RGSCA(x2, filters=64)
x2 = DeeperAttentionLayer(units=64, use_scale=True)(x2) ## GSCA layer

x2 = RGSCA(x2, filters=128, strides=(2, 2), use_projection=True)
x2 = DeeperAttentionLayer(units=128, use_scale=True)(x2)

x2 = RGSCA(x2, filters=128)
x2 = tf.keras.layers.Dropout(0.2)(x2)

x2 = RGSCA(x2, filters=128)
x2 = DeeperAttentionLayer(units=128, use_scale=True)(x2) ## GSCA layer

x2 = RGSCA(x2, filters=256, strides=(2, 2), use_projection=True)
x2 = DeeperAttentionLayer(units=256, use_scale=True)(x2)

x2 = RGSCA(x2, filters=256)
x2 = tf.keras.layers.Dropout(0.2)(x2)

x2 = RGSCA(x2, filters=256)
x2 = DeeperAttentionLayer(units=256, use_scale=True)(x2) ## GSCA layer

x2 = RGSCA(x2, filters=512, strides=(2, 2), use_projection=True)
x2 = DeeperAttentionLayer(units=512, use_scale=True)(x2)

x2 = RGSCA(x2, filters=512)
x2 = DeeperAttentionLayer(units=512, use_scale=True)(x2)

# Global average pooling and fully connected layer
con = tf.keras.layers.Concatenate(axis=-1)([x, x2])

x = GlobalAveragePooling2D()(con)
x2 = tf.keras.layers.GlobalMaxPooling2D()(con)

conx = tf.keras.layers.Add()([x, x2])
conx = tf.keras.layers.ReLU()(conx)

##x = Dense(128, activation='selu')(x)
outputs1 = Dense(3, activation='softmax')(conx)
outputs2 = Dense(2, activation='sigmoid')(conx)

# Create the model
teacher_model2 = Model([inputs, inputs2], [outputs1, outputs2])
#return model

# Instantiate the ResNet-18 model
#model = build_resnet18()


#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model
#model.fit(images_train, y_train_one_hot, epochs=10, batch_size=32, validation_split=0.2)
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
def checkpoint_callback():

    checkpoint_filepath = 'best_models_teacer_2.keras'

    model_checkpoint_callback= ModelCheckpoint(filepath=checkpoint_filepath,
                           save_weights_only=False,
                           #frequency='epoch',
                           monitor='val_loss',
                           save_best_only=True,
                           verbose=1)

    return model_checkpoint_callback

def early_stopping(patience):
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, verbose=1)
    return es_callback



checkpoint_callback = checkpoint_callback()
early_stopping = early_stopping(patience=100)
callbacks = [checkpoint_callback, early_stopping]
#model.summary()
teacher_model2.compile(optimizer='adam', loss=['categorical_crossentropy','binary_crossentropy'], 
                                      metrics=['accuracy', 'accuracy'])

'''
X_val_covid_cxr.shape, X_test_covid_cxr.shape, y_val_covid_cxr.shape, y_test_covid_cxr.shape
X_test_covid_cxr.shape, y_test_covid_cxr.shape
X_test_covid_cxr.shape, y_test_covid_cxr.shape
images_train_covid_cxr.shape, labels_train_covid_cxr.shape
X_val_covid_ct.shape, X_test_covid_ct.shape, y_val_covid_ct.shape, y_test_covid_ct.shape
X_test_covid_ct.shape, y_test_covid_ct.shape
images_train_covid_ct.shape, labels_train_covid_ct.shape
X_train_covid_ct.shape, X_test_covid_ct.shape, y_train_covid_ct.shape, y_test_covid_ct.shape

'''
print('images_train_covid_cxr:', images_train_covid_cxr.shape)
print('labels_train_covid_cxr:', labels_train_covid_cxr.shape)

print('images_train_covid_ct:', images_train_covid_ct.shape)
print('labels_train_covid_ct:', labels_train_covid_ct.shape)

print('X_val_covid_cxr:', X_val_covid_cxr.shape)
print('y_val_covid_cxr:', y_val_covid_cxr.shape)

print('X_test_covid_cxr:', X_test_covid_cxr.shape)
print('y_test_covid_cxr:', y_test_covid_cxr.shape)

print('X_val_covid_ct:', X_val_covid_ct.shape)
print('y_val_covid_ct:', y_val_covid_ct.shape)

print('X_test_covid_ct:', X_test_covid_ct.shape)
print('y_test_covid_ct:', y_test_covid_ct.shape)


teacher_model2.fit(
    [images_train_covid_cxr, images_train_covid_ct], [labels_train_covid_cxr, labels_train_covid_ct],
    epochs=200,
    validation_data=([X_val_covid_cxr, X_val_covid_ct], [y_val_covid_cxr, y_val_covid_ct]),
    callbacks=callbacks,
    #save_freq='epoch'  # This specifies saving the model after each epoch
)


images_train_covid_cxr: (5712, 128, 128, 3)
labels_train_covid_cxr: (5712, 3)
images_train_covid_ct: (5712, 128, 128, 3)
labels_train_covid_ct: (5712, 2)
X_val_covid_cxr: (655, 128, 128, 3)
y_val_covid_cxr: (655, 3)
X_test_covid_cxr: (656, 128, 128, 3)
y_test_covid_cxr: (656, 3)
X_val_covid_ct: (655, 128, 128, 3)
y_val_covid_ct: (655, 2)
X_test_covid_ct: (656, 128, 128, 3)
y_test_covid_ct: (656, 2)
Epoch 1/200


I0000 00:00:1710224649.075590      84 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


178/179 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - dense_62_accuracy: 0.7303 - dense_63_accuracy: 0.6898 - loss: 2.3645

2024-03-12 06:28:07.307515: E external/local_xla/xla/service/slow_operation_alarm.cc:65] 
********************************
[Compiling module a_inference_one_step_on_data_135179__.85706] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************
2024-03-12 06:32:44.228234: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 6m36.920950832s

********************************
[Compiling module a_inference_one_step_on_data_135179__.85706] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************


179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - dense_62_accuracy: 0.7310 - dense_63_accuracy: 0.6903 - loss: 2.3585   
Epoch 1: val_loss improved from inf to 9.10044, saving model to best_models_teacer_2.keras
179/179 ━━━━━━━━━━━━━━━━━━━━ 786s 3s/step - dense_62_accuracy: 0.7315 - dense_63_accuracy: 0.6909 - loss: 2.3526 - val_dense_62_accuracy: 0.4076 - val_dense_63_accuracy: 0.4824 - val_loss: 9.1004
Epoch 2/200
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - dense_62_accuracy: 0.9266 - dense_63_accuracy: 0.8573 - loss: 0.6054
Epoch 2: val_loss improved from 9.10044 to 2.59662, saving model to best_models_teacer_2.keras
179/179 ━━━━━━━━━━━━━━━━━━━━ 28s 158ms/step - dense_62_accuracy: 0.9266 - dense_63_accuracy: 0.8574 - loss: 0.6050 - val_dense_62_accuracy: 0.6214 - val_dense_63_accuracy: 0.7160 - val_loss: 2.5966
Epoch 3/200
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - dense_62_accuracy: 0.9457 - dense_63_accuracy: 0.9099 - loss: 0.3918
Epoch 3: val_loss did not improve from 2.59662
179/179

In [36]:
#[X_val_covid_cxr, X_val_covid_ct], [y_val_covid_cxr, y_val_covid_ct]
teacher_model2.evaluate([X_test_covid_cxr, X_test_covid_ct], [y_test_covid_cxr, y_test_covid_ct])

21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - dense_62_accuracy: 0.9757 - dense_63_accuracy: 0.9863 - loss: 0.1750


[0.21335290372371674, 0.9771341681480408, 0.9878048896789551]

In [40]:
#[X_val_covid_cxr, X_val_covid_ct], [y_val_covid_cxr, y_val_covid_ct]
teacher_model2.evaluate([X_test_covid_cxr, X_test_covid_ct], [y_test_covid_cxr, y_test_covid_ct])

21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - dense_62_accuracy: 0.9757 - dense_63_accuracy: 0.9863 - loss: 0.1750


[0.21335290372371674, 0.9771341681480408, 0.9878048896789551]

In [39]:
'''
    [images_train_covid_cxr, images_train_covid_ct], [labels_train_covid_cxr, labels_train_covid_ct],
    epochs=200,
    validation_data=([X_val_covid_cxr, X_val_covid_ct], [y_val_covid_cxr, y_val_covid_ct]),
    [X_test_covid_cxr, X_test_covid_ct], [y_test_covid_cxr, y_test_covid_ct]
'''
np.save('images_train_covid_cxr.npy', images_train_covid_cxr)
np.save('images_train_covid_ct.npy', images_train_covid_ct)
np.save('labels_train_covid_cxr.npy', labels_train_covid_cxr)
np.save('labels_train_covid_ct.npy', labels_train_covid_ct)

np.save('X_val_covid_cxr.npy', X_val_covid_cxr)
np.save('X_val_covid_ct.npy', X_val_covid_ct)
np.save('y_val_covid_cxr.npy', y_val_covid_cxr)
np.save('y_val_covid_ct.npy', y_val_covid_ct)

np.save('X_test_covid_cxr.npy', X_test_covid_cxr)
np.save('X_test_covid_ct.npy', X_test_covid_ct)
np.save('y_test_covid_cxr.npy', y_test_covid_cxr)
np.save('y_test_covid_ct.npy', y_test_covid_ct)

In [ ]:
import numpy as np
X_test_covid_ct = np.load('/kaggle/input/for-teacher-2-covid-ct-scan-and-cxr-data/X_test_covid_ct.npy')
X_test_covid_cxr = np.load('/kaggle/input/for-teacher-2-covid-ct-scan-and-cxr-data/X_test_covid_cxr.npy')
X_val_covid_ct = np.load('/kaggle/input/for-teacher-2-covid-ct-scan-and-cxr-data/X_val_covid_ct.npy')
X_val_covid_cxr = np.load('/kaggle/input/for-teacher-2-covid-ct-scan-and-cxr-data/X_val_covid_cxr.npy')

images_train_covid_ct = np.load('/kaggle/input/for-teacher-2-covid-ct-scan-and-cxr-data/images_train_covid_ct.npy')
images_train_covid_cxr = np.load('/kaggle/input/for-teacher-2-covid-ct-scan-and-cxr-data/images_train_covid_cxr.npy')
labels_train_covid_ct = np.load('/kaggle/input/for-teacher-2-covid-ct-scan-and-cxr-data/labels_train_covid_ct.npy')
labels_train_covid_cxr = np.load('/kaggle/input/for-teacher-2-covid-ct-scan-and-cxr-data/labels_train_covid_cxr.npy')

y_test_covid_ct = np.load('/kaggle/input/for-teacher-2-covid-ct-scan-and-cxr-data/y_test_covid_ct.npy')
y_test_covid_cxr = np.load('/kaggle/input/for-teacher-2-covid-ct-scan-and-cxr-data/y_test_covid_cxr.npy')
y_val_covid_ct = np.load('/kaggle/input/for-teacher-2-covid-ct-scan-and-cxr-data/y_val_covid_ct.npy')
y_val_covid_cxr = np.load('/kaggle/input/for-teacher-2-covid-ct-scan-and-cxr-data/y_val_covid_cxr.npy')

print('X_test_covid_ct:',X_test_covid_ct.shape)
print('X_test_covid_cxr:',X_test_covid_cxr.shape)
print('X_val_covid_ct:',X_val_covid_ct.shape)
print('X_val_covid_cxr:',X_val_covid_cxr.shape)

print('images_train_covid_ct:',images_train_covid_ct.shape)
print('images_train_covid_cxr:',images_train_covid_cxr.shape)
print('labels_train_covid_ct:',labels_train_covid_ct.shape)
print('labels_train_covid_cxr:',labels_train_covid_cxr.shape)

print('y_test_covid_ct:',y_test_covid_ct.shape)
print('y_test_covid_cxr:',y_test_covid_cxr.shape)
print('y_val_covid_ct:',y_val_covid_ct.shape)
print('y_val_covid_cxr:',y_val_covid_cxr.shape)

## Generating soft predictions of tasks 3 and 4 for student model by the teacher2

In [44]:
y_pred = teacher_model2.predict([images_train_covid_cxr, images_train_covid_ct])
#[images_train, images_train_ct], [labels_train_mri, labels_train_ct]
y_pred_binary = y_pred[0] >= 0.5
soft_pred_train_covid_cxr = np.array(y_pred_binary, dtype='int32')

## generating soft probabilities for task 1 of teacher 1: MRI images for binary class classification
## for student model

print('soft_pred_train_covid_cxr:', soft_pred_train_covid_cxr.shape)

## original lab=bels for MRI images
y_train_categorical = labels_train_covid_cxr
## original lab=bels for Ct scan images
y_train_categorical_c = labels_train_covid_ct

accuracy = accuracy_score(soft_pred_train_covid_cxr, y_train_categorical) * 100
precision = precision_score(soft_pred_train_covid_cxr, y_train_categorical, average='macro') * 100
recall = recall_score(soft_pred_train_covid_cxr, y_train_categorical, average='macro') * 100
f1 = f1_score(soft_pred_train_covid_cxr, y_train_categorical, average='macro') * 100
#auc = roc_auc_score(y_pred, y_train_categorical, multi_class='ovr') * 100
print('accuracy:', accuracy)
print('precision:', precision)
print('recall:', recall)
print('f1:', f1)

y_pred_binary = y_pred[1] >= 0.5
y_pred_covid_ctscan = np.array(y_pred_binary, dtype='int32')

## generating soft probabilities for task 2 of teacher 1: CT scan images for binary class classification
## for student model
print('y_pred_covid_ctscan:', y_pred_covid_ctscan.shape)
# Calculate evaluation metrics for the current epsilon
accuracy = accuracy_score(y_pred_covid_ctscan, y_train_categorical_c) * 100
precision = precision_score(y_pred_covid_ctscan, y_train_categorical_c, average='macro') * 100
recall = recall_score(y_pred_covid_ctscan, y_train_categorical_c, average='macro') * 100
f1 = f1_score(y_pred_covid_ctscan, y_train_categorical_c, average='macro') * 100
print('accuracy:', accuracy)
print('precision:', precision)
print('recall:', recall)
print('f1:', f1)


179/179 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step
soft_pred_train_covid_cxr: (5712, 3)
accuracy: 100.0
precision: 100.0
recall: 100.0
f1: 100.0
y_pred_covid_ctscan: (5712, 2)
accuracy: 100.0
precision: 100.0
recall: 100.0
f1: 100.0


In [46]:
np.save('y_pred_covid_ctscan.npy', y_pred_covid_ctscan)
np.save('soft_pred_train_covid_cxr.npy', soft_pred_train_covid_cxr)

In [ ]:
y_pred_covid_ctscan, soft_pred_train_covid_cxr

In [37]:
teacher_model2.save('best_ever_teacher2_model.keras')

In [35]:
from tensorflow.keras.models import load_model

# Load the model
teacher_model2_1 = load_model('/kaggle/working/best_models_teacer_2.keras', 
                              custom_objects={'DeeperAttentionLayer':DeeperAttentionLayer})
teacher_model2_1.evaluate([X_test_covid_cxr, X_test_covid_ct], [y_test_covid_cxr, y_test_covid_ct])
               #batch_size=128)

21/21 ━━━━━━━━━━━━━━━━━━━━ 24s 406ms/step - dense_62_accuracy: 0.9556 - dense_63_accuracy: 0.9838 - loss: 0.2089


[0.23082010447978973, 0.9634146094322205, 0.9847561120986938]